# ScopeSim + IRDB validation for MICADO

This is referencing Ric Davies' document *MICADO Imaging and Spectroscopy Reference*, Doc. No.: ELT-TRE-MCD-56300-0173, Issue: 0.5, Date: 14.06.2023. This document is hereinafter referred to as "the reference document". Any mention of sections, tables and figures refer to those listed in that document.

In [ ]:
import scopesim as sim
from matplotlib import pyplot as plt

In [ ]:
# sim.download_packages(["Armazones", "ELT", "MORFEO", "MICADO"], release="latest")

In [ ]:
cmds = sim.UserCommands(use_instrument="MICADO", set_modes=["SCAO", "SPEC_3000x16"])
cmds.cmds["!OBS.filter_name_fw1"] = "Spec_IJ"
cmds.cmds["!OBS.filter_name_fw2"] = "open"
micado = sim.OpticalTrain(cmds)

# 3. Optical Train & Transmission

### General info about the optical train object

Note e.g. the `!INST.plate_scale`, which matches with the final plate scale for spectroscopy mode given in **Table 3**.

In [ ]:
micado

### Specific info about the included effects

This represents the optical elements, among other effects, and is related to **Table 1** and **Figure 1**. Some parts, like sequences of mirrors, are combined into a single effect type (e.g. `SurfaceList`).

Using the `.show_in_notebook()` method to get a non-truncated table.

In [ ]:
micado.effects.show_in_notebook()

# 4. Focal Plane Array and Detector Characteristics

## 4.1 Detector and Pixel Numbering

### Figure 3

This is a recreation of the spectral traces on the detector layout, using the actual `OpticalTrain` object in ScopeSim.
Note the orientation is flipped compared to the reference document. Some orders (8_3, 5_2) are missing, as they are missing from the IRDB.

In [ ]:
# Setup figure
fig, axes = plt.subplots(1, 3, figsize=(12, 4))

modes = ["SPEC_15000x20", "SPEC_15000x20", "SPEC_3000x16"]
filters = ["Spec_HK", "J", "Spec_IJ"]

for mode, fltr, ax in zip(modes, filters, axes):
    cmds = sim.UserCommands(use_instrument="MICADO", set_modes=["SCAO", mode])
    cmds.cmds["!OBS.filter_name_fw1"] = fltr
    cmds.cmds["!OBS.filter_name_fw2"] = "open"
    micado = sim.OpticalTrain(cmds)

    tbl = micado["filter_wheel_1"].current_filter.table
    wav_min = tbl[tbl["transmission"] > 0.001]["wavelength"].min()
    wav_max = tbl[tbl["transmission"] > 0.0]["wavelength"].max()
    xi_min = micado["spectroscopic_slit_aperture"].table["x"].min()
    xi_max = micado["spectroscopic_slit_aperture"].table["x"].max()

    micado["full_detector_array"].plot(axes=ax)
    micado["micado_spectral_traces"].plot(wave_min=wav_min, wave_max=wav_max, xi_min=xi_min, xi_max=xi_max, axes=ax,
                                      plot_footprint=False, plot_wave=False, plot_ctrlpnts=False, plot_outline=True,
                                      plot_trace_id=True)
    ax.set_title(f"{fltr=}, {mode=}")
fig.tight_layout()

## 4.2 Detector properties

### Table 4

The positions and angles of the individual detectors. Table is transposed compared to the one in the reference document.

In [ ]:
micado["full_detector_array"].table[["id", "x_cen", "y_cen", "angle"]]

### QE curve

The following plot shows the detector QE, which is related to the information in **Table 5**.

In [ ]:
micado["qe_curve"].surface.throughput.plot()

# 5. Focal Plane Mapping

# 6. Imaging Characteristics (LRI, HRI)

# 7. Spectral traces

## 7.2 Grating Efficiencies
### Figure 10

In [ ]:
micado["grating_efficiency"].plot().show()

# 8. Pupil Imaging (PUP)

# 9. Raw Data Format and Keywords